In [2]:
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [ ]:
class StoryTime:
    def __init__(self):
        super(StoryTime, self).__init__()
        self.device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer  = GPT2Tokenizer.from_pretrained("gpt2")
        self.model      = GPT2LMHeadModel.from_pretrained("gpt2")

    def get_next_word(self, text):
        '''Get the next top word prediction based on text.'''
        indexed_tokens = self.tokenizer.encode(text)
        tokens_tensor  = torch.tensor([indexed_tokens])

        self.model.eval()
        tokens_tensor = tokens_tensor.to(self.device)
        self.model.to(self.device)
        
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]

        probs    = predictions[0, -1, :]
        top_next = [self.tokenizer.decode(i.item()).strip() for i in probs.topk(10)[1]]
        old_text = text.split()
        
        for item in top_next:
            if item != old_text[-1]:
                return item
    
    def get_next_n_words(self, curr_text, n):
        '''Get the next n words for curr_text'''
        for i in range(n):
            next_word = self.get_next_word(curr_text)
            curr_text = curr_text + " " + next_word
            
        return curr_text

In [ ]:
st = StoryTime()

In [ ]:
curr_text = ""
while(True):
    inp = input("Input: ")
    curr_text = curr_text + " " + inp
    curr_text = st.get_next_n_words(curr_text, 3)
    print("\nOutput: " + curr_text + "\n")